In [ ]:
!ls /content/drive/MyDrive/ECE_542/Competition_Project

data.zip  metadata  ml_utils


In [ ]:
!cp /content/drive/MyDrive/ECE_542/Competition_Project/data.zip .

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/.gitignore         
   creating: data/preprocessed_data/
   creating: data/splits/
   creating: data/splits/test/
  inflating: data/splits/test/subject_005_session_03__x.csv  
  inflating: data/splits/test/subject_005_session_03__y.csv  
  inflating: data/splits/test/subject_006_session_01__x.csv  
  inflating: data/splits/test/subject_006_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_02__x.csv  
  inflating: data/splits/test/subject_006_session_02__y.csv  
  inflating: data/splits/test/subject_006_session_03__x.csv  
  inflating: data/splits/test/subject_006_session_03__y.csv  
  inflating: data/splits/test/subject_007_session_01__x.csv  
  inflating: data/splits/test/subject_007_session_01__y.csv  
  inflating: data/splits/test/subject_007_session_02__x.csv  
  inflating: data/splits/test/subject_007_session_02__y.csv  
  inflating: data/splits/test/subject_007_session_03__x.csv  
  inflating: data/splits

In [ ]:
!rm data.zip

rm: cannot remove 'data.zip': No such file or directory


In [ ]:
!cp -r /content/drive/MyDrive/ECE_542/Competition_Project/ml_utils .
!ls

data  data.zip	drive  ml_utils  sample_data


In [ ]:
base_path = "/content/drive/MyDrive/ECE_542/Competition_Project"

In [ ]:
import os
import sys
# sys.path.append(base_path)

In [ ]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from ml_utils.dataset import SubjectDataset
from collections import Counter
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda:0


In [ ]:
base_data_path = os.path.join("data", "splits")
train_data_path = os.path.join(base_data_path, "train")
val_data_path = os.path.join(base_data_path, "val")

splits_file = os.path.join(base_path, "metadata", "split_ids.json")
with open(splits_file, "r") as f:
    split_ids = json.load(f)

In [ ]:
# Training hyperparameters
batch_size = 256
num_epochs = 5

In [ ]:
train_dataset = SubjectDataset(
    train_data_path, 
    split_ids["train"], 
    cache_len=len(split_ids["train"])
)
ys = train_dataset.index_store["label"].to_list()
counts = Counter(ys)
weights = np.array([1./counts[_y] for _y in ys])
sample_weights = torch.from_numpy(weights).float()
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_iterations = (len(train_dataset) // batch_size) + ((len(train_dataset) % batch_size) != 0)

val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"], 
    cache_len=len(split_ids["val"])
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

In [ ]:
print(train_iterations)
print(val_iterations)

687
211


In [ ]:
class OneDConvNet(nn.Module):
  def __init__(self, n_features, n_classes):
    super(OneDConvNet, self).__init__()
    self.Convolution_Layer_1 = nn.Conv1d(in_channels=n_features, out_channels=8, kernel_size=3, stride=1)
    self.Pool_Layer1         = nn.MaxPool1d(kernel_size=2, stride=2)
    self.Convolution_Layer_2 = nn.Conv1d(in_channels= 8, out_channels=16, kernel_size=3, stride=1)
    self.Pool_Layer2         = nn.MaxPool1d(kernel_size=2, stride=2)
    self.Convolution_Layer_3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
    self.Pool_Layer3         = nn.MaxPool1d(kernel_size=2, stride=2)
    self.FullConnected1      = nn.Linear(32*3,16)
    self.FullConnected2      = nn.Linear(16,8)
    self.FullConnected3      = nn.Linear(8, n_classes)
  
  def forward(self, x):
    x = self.Pool_Layer1(F.relu(self.Convolution_Layer_1(x)))
    x = self.Pool_Layer2(F.relu(self.Convolution_Layer_2(x)))
    x = self.Pool_Layer3(F.relu(self.Convolution_Layer_3(x)))
    N, C, T = x.size()
    x = x.view(-1, C*T) # Flatten
    x = F.relu(self.FullConnected1(x))
    x = F.relu(self.FullConnected2(x))
    x = self.FullConnected3(x)
    return x

model = OneDConvNet(6, 4).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_step(X, y, model, optimizer, criterion):

    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [ ]:
for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        y = y.view(X.size(0)).to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss * X.size(0)
        running_acc = running_acc + corrects

        if iteration % 100 == 0:
            print(f"Iteration: {iteration}/{train_iterations} | train_loss: {loss} | train_acc: {corrects/X.size(0)}")

    train_loss = running_loss / len(train_dataset)
    train_acc = running_acc / len(train_dataset)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        y = y.view(X.size(0)).to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss * X.size(0)
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataset)
    val_acc = running_val_acc / len(val_dataset)

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Iteration: 0/687 | train_loss: 1.428921103477478 | train_acc: 0.2265625
Iteration: 100/687 | train_loss: 1.0766388177871704 | train_acc: 0.5
Iteration: 200/687 | train_loss: 0.6234880089759827 | train_acc: 0.75
Iteration: 300/687 | train_loss: 0.45234811305999756 | train_acc: 0.79296875
Iteration: 400/687 | train_loss: 0.47503662109375 | train_acc: 0.76953125
Iteration: 500/687 | train_loss: 0.36301949620246887 | train_acc: 0.85546875
Iteration: 600/687 | train_loss: 0.40121737122535706 | train_acc: 0.84375
Epoch: 0 | train_loss 0.6350234015076079 | train_acc: 0.7053002715110779 | val_loss: 0.8903307719343264 | val_acc: 0.6590913534164429
Iteration: 0/687 | train_loss: 0.3501107692718506 | train_acc: 0.8515625
Iteration: 100/687 | train_loss: 0.3713786005973816 | train_acc: 0.81640625
Iteration: 200/687 | train_loss: 0.43050146102905273 | train_acc: 0.859375
Iteration: 300/687 | train_loss: 0.4585791826248169 | train_acc: 0.84375
Iteration: 400/687 | train_loss: 0.37795084714889526 | t